In [ ]:
# stp 1 Dependecis instal
#!pip install transformers==4.41.2
# Important to install that transformer version, unles it occur error
#See link: https://github.com/THUDM/CogVLM2/issues/181#issuecomment-2381807778
# !pip install transformers sentencepiece datasets evaluate
# !pip install llmlingua

In [1]:
import torch
import json
import time
import os
from transformers import AutoModel, AutoTokenizer, GenerationConfig
from datasets import load_dataset
from tqdm.notebook import tqdm 
import numpy as np
from sklearn.metrics import f1_score
#from LLMLingua.experiments.llmlingua2.evaluation.metrics import qa_f1_score
from metrics import qa_f1_score, f1_score
if 'model' in globals():
    del model 
torch.cuda.empty_cache()

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#######################################
print('cuda is',torch.cuda.is_available()) 
#################################################
# 3 model & dataset 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda is True


In [ ]:
print('cuda is',torch.cuda.is_available()) 

In [2]:
# ChatGLM-6B
model = AutoModel.from_pretrained(
    "THUDM/chatglm2-6b-32k",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True
).to(device)

model=model.eval()
#print('model config is:',model.generation_config)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True)#.to(device)

In [4]:
# Monkey-patch the tokenizer's _pad method to handle the padding_side argument
original_pad = tokenizer._pad
def new_pad(self, encoded_inputs, max_length=None, padding_strategy="longest", pad_to_multiple_of=None, return_attention_mask=None, **kwargs):
    return original_pad(encoded_inputs, max_length, padding_strategy, pad_to_multiple_of, return_attention_mask)
tokenizer._pad = new_pad.__get__(tokenizer, type(tokenizer))


In [5]:
# Load the "multifieldqa_en" subset
dataset_name = "multifieldqa_en"
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")

# print(type(data))#should be <class 'datasets.arrow_dataset.Dataset'>
# print(data[0])  #keys like "context", "question", "answers"

In [ ]:
#RANDOM token PRUNE
def random_prune(text, compression_ratio=0.5):

    tokens = tokenizer.tokenize(text)#.to(device)  # e.g., ["The", " quick", " brown"...]
    
    num_keep = int(len(tokens) * (compression_ratio)) # compression ratio means remaining 
    
    # Step 3: Randomly pick tokens (like lottery balls)
    kept_indices = sorted(  # Maintain original order
        np.random.choice(  # Random selection
            len(tokens), 
            num_keep, 
            replace=False  # No duplicates
        )
    )
    
    # Step 4: Rebuild text from kept tokens
    pruned_tokens = [tokens[i] for i in kept_indices]
    return tokenizer.convert_tokens_to_string(pruned_tokens)  # Tokens ➔ text

In [6]:
# LINGUA COMPRESSION

from llmlingua import PromptCompressor

def compress_with_llmlingua(context, ratio=0.5):
    compressor = PromptCompressor()#.to(device)
    compressed_context = compressor.compress_prompt(context, rate=ratio)#,device=device)
    # compressed_context = compressor.compress_prompt_llmlingua2(context, rate=ratio)
    return compressed_context

In [ ]:
# Any summarizer
from transformers import BartForConditionalGeneration, BartTokenizer
def compress_with_bart(context, ratio=0.5):
    model= BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer(
        context, 
        return_tensors="pt", 
        truncation=True, 
        max_length=1024
    ).to(device)
    # Calculate target length based on ratio
    input_length = inputs['input_ids'].shape[1]
    target_length = int(input_length * ratio)
    
    # Generate summary
    summary_ids = model.generate(
        inputs.input_ids,
        num_beams=4,
        max_length=target_length,
        min_length=max(target_length - 100, 50),  # Keep min_length within bounds
        length_penalty=2.0,
        early_stopping=True
    )
    
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
#Baseline

def no_pruning(context: str) -> str:
    """Returns the full context (no pruning)."""
    return context


In [7]:
def evaluate(data, compression_func, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []
    
    #for idx in range(10):  # Test on 10 examples
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        #print(type(example))
        #print(example)
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        if ratio is not None:
            compressed_context = compression_func(context, ratio)
        else:
            compressed_context = compression_func(context)  #for no.prune & compressr
        
        # test_response, _ = model.chat(
        # tokenizer,
        # "What is the capital of France?",
        # history=[("European geography", "Answer:")]
        # )
        # print(test_response)  # Should return "Paris"
            
        prompt = f"Context: {compressed_context}\nQuestion: {question}"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        #f1 = f1_score([answers], [response], average="macro")  # Stored f1score function is not appropritate
        #f1=compute_f1(answers, response)
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        #f1 = f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        memory_usages.append(memory)
        peak_memory_usages.append(peak_memory)

        # print(f"Response: {response}")
        # print(f"F1 Score: {f1:.2f}, Latency: {latency:.2f}s, Memory: {memory:.2f}MB\n")
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        #"avg_memory": np.mean(memory_usages),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [8]:
# Reusable function to print and save results
def log_result_no_prune(message):
    #print(message)
    with open("results_no_prune.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

def log_result_random_prune(message):
    #print(message)
    with open("results_random_prune.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

def log_result_BART(message):
    #print(message)
    with open("results_BART.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")
        
def log_result_LLMLingua(message):
    #print(message)
    with open("results_LLMLingua.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

In [ ]:
# Evaluate no pruning (full context)
full_context_results = evaluate(data, no_pruning)
print("\n=== No Pruning (Full Context) ===")
print(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s,  Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")
log_result_no_prune("\n=== No Pruning (Full Context) ===")
log_result_no_prune(f"F1: {full_context_results['avg_f1']:.2f}, Latency: {full_context_results['avg_latency']:.2f}s, Peak Memory: {full_context_results['avg_peak_memory']:.2f} MB")

In [ ]:
pruning_ratios = [0.5, 0.6, 0.7]

for ratio in pruning_ratios:
    bart_results = evaluate(data, compress_with_bart, ratio=ratio)
    
    print(f"\n=== bart compression ({int(ratio*100)}%) ===")
    print(f"F1: {bart_results['avg_f1']:.2f}, Latency: {bart_results['avg_latency']:.2f}s,  Peak Memory: {bart_results['avg_peak_memory']:.2f} MB")
    log_result_BART(f"\n=== bart compression ({int(ratio*100)}%) ===")
    log_result_BART(f"F1: {bart_results['avg_f1']:.2f}, Latency: {bart_results['avg_latency']:.2f}s,  Peak Memory: {bart_results['avg_peak_memory']:.2f} MB")
        

In [9]:
pruning_ratios = [0.5, 0.6, 0.7]
for ratio in pruning_ratios:
    lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    
    print(f"\n=== llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n=== llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")

/home/seongjae/miniconda3/envs/seongjae_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


=== llmlingua compression (70%) ===
F1: 0.28, Latency: 1.41s,  Peak Memory: 12745.41 MB


In [10]:
pruning_ratios = [0.5, 0.6, 0.7]
for ratio in pruning_ratios:
    random_pruning_results = evaluate(data, random_prune, ratio=ratio)
    
    print(f"\n=== Random Token Pruning ({int(ratio*100)}%) ===")
    print(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")
    log_result_random_prune(f"\n=== Random Token Pruning ({int(ratio*100)}%) ===")
    log_result_random_prune(f"F1: {random_pruning_results['avg_f1']:.2f}, Latency: {random_pruning_results['avg_latency']:.2f}s, Peak Memory: {random_pruning_results['avg_peak_memory']:.2f} MB")

NameError: name 'random_prune' is not defined